In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.datasets import load_wine
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("Modelar_UH2020.txt", sep = "|")

df.dropna(inplace = True)
print(f"Nº de resgistros nulos: {df.isnull().sum().max()}")
print("\n")

df["CADASTRALQUALITYID"].replace("A", 10, inplace = True)
df["CADASTRALQUALITYID"].replace("B", 11, inplace = True)
df["CADASTRALQUALITYID"].replace("C", 12, inplace = True)

print(f"Valores únicos en CADASTRALQUALITYID: {df.CADASTRALQUALITYID.unique()}")

print("\n")
print(f"CLASES: {df.CLASE.unique()}")
df.CLASE = pd.Categorical(df.CLASE)
df.CLASE = df.CLASE.cat.codes
print(f"CLASE en numérico: {df.CLASE.unique()}")
print("\n")
#Chequeo de "no balanceo"
print("Nº de observaciones por CLASE")
print(df["CLASE"].value_counts())

df.iloc[:,1:52]=df.iloc[:,1:52].astype("float32")
df.iloc[:,52] = df.iloc[:,52].astype("int32")
df.iloc[:,53] = df.iloc[:,53].astype("float32")
df.iloc[:,54] = df.iloc[:,54].astype("int8")

min_con= df["CONTRUCTIONYEAR"].min()
df["CONTRUCTIONYEAR"] = df["CONTRUCTIONYEAR"] - min_con

Nº de resgistros nulos: 0


Valores únicos en CADASTRALQUALITYID: ['2' '8' '5' '6' '4' '7' '3' 12 '1' '9' 11 10]


CLASES: ['RESIDENTIAL' 'INDUSTRIAL' 'PUBLIC' 'OFFICE' 'OTHER' 'RETAIL'
 'AGRICULTURE']
CLASE en numérico: [5 1 4 2 3 6 0]


Nº de observaciones por CLASE
5    90173
1     4486
4     2976
6     2092
2     1828
3     1332
0      323
Name: CLASE, dtype: int64


In [3]:
df_dist = pd.read_csv("modelar_distancias.csv")
df_dist = df_dist[["ID","Xpred","Ypred", "1000mresidential","1000mindustrial","1000mpublic","1000moffice","1000mother","1000mretail","1000magriculture",
                   "500mresidential","500mindustrial","500mpublic","500moffice","500mother","500mretail","500magriculture",
                   "250mresidential","250mindustrial","250mpublic","250moffice","250mother","250mretail","250magriculture",
                   "100mresidential","100mindustrial","100mpublic","100moffice","100mother","100mretail","100magriculture"]]
df_dist.iloc[:,1:3] = df_dist.iloc[:,1:3].astype("float32")
df_dist.iloc[:,3:31] = df_dist.iloc[:,3:31].astype("int8")
#df_dist.info()

In [4]:
df = pd.merge(df, df_dist, on = "ID")
df.head()

,ID,X,Y,Q_R_4_0_0,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
0,35984B9C3E7CD9A1,2.207358e+09,165920304.0,0.000000,443.001312,616.001709,746.998413,872.996460,1009.000916,1159.002319,...,0,0,0,9,0,0,0,0,0,0
1,F9D04BF6D037F8FB,2.189757e+09,165463264.0,0.000059,443.899017,627.999084,770.001587,905.000000,1032.998413,1165.001587,...,0,1,0,48,0,0,0,0,0,0
2,B89D5711AFF8C423,2.240147e+09,165690752.0,0.000000,353.502258,523.003601,644.001831,760.997131,876.999634,1006.997498,...,0,1,0,13,0,0,0,0,0,0
3,1C3478AC1522E7E4,2.227146e+09,165934096.0,0.000000,268.000610,376.999603,478.003784,575.001221,683.997742,809.005981,...,0,0,0,8,0,1,0,0,0,0
4,4D12AA5009064345,2.212350e+09,165681792.0,0.000000,318.997894,492.003845,632.999634,757.002197,882.999939,1019.008911,...,0,0,0,2,0,0,0,0,0,0


In [5]:
df.drop(df[["Q_R_4_0_0","Q_R_4_1_0","Q_G_3_0_0","Q_G_3_1_0","Q_B_2_0_0","Q_B_2_1_0","Q_NIR_8_0_0","Q_NIR_8_1_0"]], axis = "columns", inplace = True)
df.drop(df[["ID"]], axis = "columns", inplace = True)

In [6]:
df0 = df[df["CLASE"]==0]
df0.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
322,2.197578e+09,165936160.0,547.999695,658.000732,744.002563,825.000061,905.998535,993.999573,1105.999512,1266.000977,...,0,0,4,0,0,0,0,0,0,3
917,2.217737e+09,165974592.0,262.999878,419.999390,576.000427,714.997864,853.998657,1014.995728,1211.997314,1482.999390,...,0,0,1,0,0,0,0,0,0,1
975,2.276415e+09,165894096.0,463.004089,659.002808,781.003479,896.000061,1017.000366,1158.995117,1335.999268,1553.000732,...,0,0,1,0,0,0,0,0,0,1
1000,2.134138e+09,165924320.0,533.001404,613.999878,670.999329,729.998718,804.000793,898.999390,1021.001404,1171.000610,...,0,0,1,0,0,0,0,0,0,1
1516,2.262635e+09,165306160.0,562.000488,720.999756,822.001953,913.001221,1024.999146,1164.000244,1337.000366,1588.998291,...,0,0,1,0,0,0,0,0,0,1


In [7]:
df0 = df0[np.abs(df0.AREA-df0.AREA.mean()) <= (3*df0.AREA.std())]
df0 = df0[np.abs(df0.GEOM_R2 - df0.GEOM_R2.mean()) <= (3*df0.GEOM_R2.std())]
df0 = df0[np.abs(df0.GEOM_R3 - df0.GEOM_R3.mean()) <= (3*df0.GEOM_R3.std())]
df0 = df0[np.abs(df0.GEOM_R4 - df0.GEOM_R4.mean()) <= (3*df0.GEOM_R4.std())]
df0 = df0[np.abs(df0.CONTRUCTIONYEAR - df0.CONTRUCTIONYEAR.mean()) <= (3*df0.CONTRUCTIONYEAR.std())]
df0 = df0[np.abs(df0.MAXBUILDINGFLOOR - df0.MAXBUILDINGFLOOR.mean()) <= (3*df0.MAXBUILDINGFLOOR.std())]
df0 = df0[np.abs(df0.CADASTRALQUALITYID - df0.CADASTRALQUALITYID.mean()) <= (3*df0.CADASTRALQUALITYID.std())]

In [8]:
df1 = df[df["CLASE"]==1]
df1.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
9,2.267211e+09,165899056.0,454.999329,573.005005,668.002625,752.999573,853.000122,1005.999146,1201.701660,1448.999146,...,0,1,0,0,10,0,0,0,0,0
48,2.200806e+09,165402352.0,443.000000,619.998291,762.994080,907.004028,1054.003052,1202.999023,1360.993896,1553.998657,...,0,0,0,14,7,0,0,0,0,0
55,2.258692e+09,165690304.0,572.003784,765.000366,924.996460,1097.998169,1329.000732,1609.997925,1974.998657,2479.006348,...,0,3,0,1,2,0,0,0,0,0
61,2.268047e+09,165912208.0,454.999329,573.005005,668.002625,752.999573,853.000122,1005.999146,1201.701660,1448.999146,...,0,1,0,0,3,0,0,0,0,0
95,2.175121e+09,165733872.0,364.700409,504.003540,612.999451,721.994873,840.997620,985.999329,1172.001343,1439.012573,...,0,0,0,4,1,0,0,0,0,0


In [9]:
df1 = df1[np.abs(df1.AREA-df1.AREA.mean()) <= (3*df1.AREA.std())]
df1 = df1[np.abs(df1.GEOM_R2 - df1.GEOM_R2.mean()) <= (3*df1.GEOM_R2.std())]
df1 = df1[np.abs(df1.GEOM_R3 - df1.GEOM_R3.mean()) <= (3*df1.GEOM_R3.std())]
df1 = df1[np.abs(df1.GEOM_R4 - df1.GEOM_R4.mean()) <= (3*df1.GEOM_R4.std())]
df1 = df1[np.abs(df1.CONTRUCTIONYEAR - df1.CONTRUCTIONYEAR.mean()) <= (3*df1.CONTRUCTIONYEAR.std())]
df1 = df1[np.abs(df1.MAXBUILDINGFLOOR - df1.MAXBUILDINGFLOOR.mean()) <= (3*df1.MAXBUILDINGFLOOR.std())]
df1 = df1[np.abs(df1.CADASTRALQUALITYID - df1.CADASTRALQUALITYID.mean()) <= (3*df1.CADASTRALQUALITYID.std())]

In [10]:
df2 = df[df["CLASE"]==2]
df2.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
72,2.230495e+09,165675840.0,217.000046,322.003448,422.001923,522.006165,629.999268,753.999207,915.004272,1144.008911,...,0,1,0,6,0,0,1,0,0,0
76,2.207499e+09,165617632.0,385.001007,580.005737,737.004456,873.996460,1005.998779,1141.001709,1296.997070,1505.996704,...,0,0,0,0,0,0,2,0,0,0
89,2.189053e+09,165277312.0,633.000183,725.999939,804.000061,886.997437,980.000610,1083.000000,1205.999390,1412.000122,...,0,0,0,0,2,0,1,0,0,0
116,2.210304e+09,165614352.0,364.999878,548.002014,696.005066,828.997742,954.005249,1098.000244,1262.999146,1489.991699,...,0,0,0,3,0,0,1,0,0,0
154,2.239326e+09,165736816.0,406.999573,574.599854,728.998535,892.999146,1056.999756,1232.997070,1456.109985,1784.997192,...,0,2,0,0,0,0,1,0,0,0


In [11]:
df2 = df2[np.abs(df2.AREA-df2.AREA.mean()) <= (3*df2.AREA.std())]
df2 = df2[np.abs(df2.GEOM_R2 - df2.GEOM_R2.mean()) <= (3*df2.GEOM_R2.std())]
df2 = df2[np.abs(df2.GEOM_R3 - df2.GEOM_R3.mean()) <= (3*df2.GEOM_R3.std())]
df2 = df2[np.abs(df2.GEOM_R4 - df2.GEOM_R4.mean()) <= (3*df2.GEOM_R4.std())]
df2 = df2[np.abs(df2.CONTRUCTIONYEAR - df2.CONTRUCTIONYEAR.mean()) <= (3*df2.CONTRUCTIONYEAR.std())]
df2 = df2[np.abs(df2.MAXBUILDINGFLOOR - df2.MAXBUILDINGFLOOR.mean()) <= (3*df2.MAXBUILDINGFLOOR.std())]
df2 = df2[np.abs(df2.CADASTRALQUALITYID - df2.CADASTRALQUALITYID.mean()) <= (3*df2.CADASTRALQUALITYID.std())]

In [12]:
df3 = df[df["CLASE"]==3]
df3.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
113,2.224149e+09,165432016.0,499.700439,678.003540,806.003235,922.999207,1040.998291,1162.994507,1308.004150,1501.994995,...,0,1,0,20,0,0,0,1,0,0
121,2.183073e+09,165515136.0,272.000000,415.999481,570.500244,726.998474,886.998840,1048.999268,1218.996460,1424.003052,...,0,1,0,2,0,1,0,1,1,0
203,2.245084e+09,165685680.0,309.000610,456.999756,577.000549,689.006165,800.003113,925.999817,1082.000610,1292.996582,...,0,0,0,4,0,0,0,2,0,0
275,2.240089e+09,165519392.0,466.700043,601.001038,719.001343,846.001221,983.999207,1124.998169,1294.999023,1504.002441,...,0,0,0,7,1,0,0,3,0,0
372,2.230857e+09,165770400.0,323.504120,487.996552,622.997009,748.000183,875.008057,1016.999634,1178.999756,1379.998047,...,0,0,0,2,0,0,0,1,0,0


In [13]:
df3 = df3[np.abs(df3.AREA-df3.AREA.mean()) <= (3*df3.AREA.std())]
df3 = df3[np.abs(df3.GEOM_R2 - df3.GEOM_R2.mean()) <= (3*df3.GEOM_R2.std())]
df3 = df3[np.abs(df3.GEOM_R3 - df3.GEOM_R3.mean()) <= (3*df3.GEOM_R3.std())]
df3 = df3[np.abs(df3.GEOM_R4 - df3.GEOM_R4.mean()) <= (3*df3.GEOM_R4.std())]
df3 = df3[np.abs(df3.CONTRUCTIONYEAR - df3.CONTRUCTIONYEAR.mean()) <= (3*df3.CONTRUCTIONYEAR.std())]
df3 = df3[np.abs(df3.MAXBUILDINGFLOOR - df3.MAXBUILDINGFLOOR.mean()) <= (3*df3.MAXBUILDINGFLOOR.std())]
df3 = df3[np.abs(df3.CADASTRALQUALITYID - df3.CADASTRALQUALITYID.mean()) <= (3*df3.CADASTRALQUALITYID.std())]

In [14]:
df4 = df[df["CLASE"]==4]
df4.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
12,2.230760e+09,165513200.0,344.003510,537.000488,678.994141,800.003784,926.997253,1070.998901,1241.997314,1467.997192,...,0,0,0,0,0,1,0,0,0,0
37,2.211011e+09,165763888.0,293.001495,457.001648,587.001526,709.998535,829.998901,962.994080,1121.996704,1335.997314,...,0,0,0,14,0,1,0,0,0,0
125,2.200157e+09,165598272.0,456.008209,650.000732,803.003662,945.997009,1080.000000,1207.002319,1348.995972,1514.994141,...,0,0,0,4,0,1,0,0,0,0
157,2.204507e+09,165567248.0,456.008209,650.000732,803.003662,945.997009,1080.000000,1207.002319,1348.995972,1514.994141,...,0,0,0,5,0,1,0,0,0,0
173,2.197077e+09,165520704.0,366.000519,545.000183,698.003967,833.996582,951.001038,1081.998169,1233.000732,1420.003906,...,0,1,0,2,0,1,0,0,0,0


In [15]:
df4 = df4[np.abs(df4.AREA-df4.AREA.mean()) <= (3*df4.AREA.std())]
df4 = df4[np.abs(df4.GEOM_R2 - df4.GEOM_R2.mean()) <= (3*df4.GEOM_R2.std())]
df4 = df4[np.abs(df4.GEOM_R3 - df4.GEOM_R3.mean()) <= (3*df4.GEOM_R3.std())]
df4 = df4[np.abs(df4.GEOM_R4 - df4.GEOM_R4.mean()) <= (3*df4.GEOM_R4.std())]
df4 = df4[np.abs(df4.CONTRUCTIONYEAR - df4.CONTRUCTIONYEAR.mean()) <= (3*df4.CONTRUCTIONYEAR.std())]
df4 = df4[np.abs(df4.MAXBUILDINGFLOOR - df4.MAXBUILDINGFLOOR.mean()) <= (3*df4.MAXBUILDINGFLOOR.std())]
df4 = df4[np.abs(df4.CADASTRALQUALITYID - df4.CADASTRALQUALITYID.mean()) <= (3*df4.CADASTRALQUALITYID.std())]

In [16]:
df5 = df[df["CLASE"]==5]
df5.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
0,2.207358e+09,165920304.0,443.001312,616.001709,746.998413,872.996460,1009.000916,1159.002319,1342.002441,1572.003418,...,0,0,0,9,0,0,0,0,0,0
1,2.189757e+09,165463264.0,443.899017,627.999084,770.001587,905.000000,1032.998413,1165.001587,1316.000122,1505.990723,...,0,1,0,48,0,0,0,0,0,0
2,2.240147e+09,165690752.0,353.502258,523.003601,644.001831,760.997131,876.999634,1006.997498,1168.995483,1375.008179,...,0,1,0,13,0,0,0,0,0,0
3,2.227146e+09,165934096.0,268.000610,376.999603,478.003784,575.001221,683.997742,809.005981,955.002930,1136.997192,...,0,0,0,8,0,1,0,0,0,0
4,2.212350e+09,165681792.0,318.997894,492.003845,632.999634,757.002197,882.999939,1019.008911,1175.993042,1401.000488,...,0,0,0,2,0,0,0,0,0,0


In [17]:
df5 = df5[np.abs(df5.AREA-df5.AREA.mean()) <= (3*df5.AREA.std())]
df5 = df5[np.abs(df5.GEOM_R2 - df5.GEOM_R2.mean()) <= (3*df5.GEOM_R2.std())]
df5 = df5[np.abs(df5.GEOM_R3 - df5.GEOM_R3.mean()) <= (3*df5.GEOM_R3.std())]
df5 = df5[np.abs(df5.GEOM_R4 - df5.GEOM_R4.mean()) <= (3*df5.GEOM_R4.std())]
df5 = df5[np.abs(df5.CONTRUCTIONYEAR - df5.CONTRUCTIONYEAR.mean()) <= (3*df5.CONTRUCTIONYEAR.std())]
df5 = df5[np.abs(df5.MAXBUILDINGFLOOR - df5.MAXBUILDINGFLOOR.mean()) <= (3*df5.MAXBUILDINGFLOOR.std())]
df5 = df5[np.abs(df5.CADASTRALQUALITYID - df5.CADASTRALQUALITYID.mean()) <= (3*df5.CADASTRALQUALITYID.std())]

In [18]:
df6 = df[df["CLASE"]==6]
df6.head()

,X,Y,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,...,250mother,250mretail,250magriculture,100mresidential,100mindustrial,100mpublic,100moffice,100mother,100mretail,100magriculture
135,2.192781e+09,165285520.0,464.999939,610.000244,721.000977,829.000000,953.000854,1122.000000,1347.999878,1621.995483,...,0,1,1,0,2,0,0,0,1,0
148,2.207601e+09,165688848.0,304.999237,513.000488,684.996521,844.999146,994.001282,1150.202759,1329.001099,1549.994995,...,0,2,0,6,0,0,1,0,1,0
254,2.227547e+09,165425104.0,497.998047,695.001404,849.997498,996.997314,1150.999390,1318.999023,1512.007324,1779.001709,...,0,2,0,1,0,0,0,2,2,0
443,2.234984e+09,165399744.0,497.998047,695.001404,849.997498,996.997314,1150.999390,1318.999023,1512.007324,1779.001709,...,0,5,0,14,0,0,1,0,1,0
446,2.232262e+09,165752016.0,323.504120,487.996552,622.997009,748.000183,875.008057,1016.999634,1178.999756,1379.998047,...,0,4,0,4,0,0,0,1,3,0


In [19]:
df6 = df6[np.abs(df6.AREA-df6.AREA.mean()) <= (3*df6.AREA.std())]
df6 = df6[np.abs(df6.GEOM_R2 - df6.GEOM_R2.mean()) <= (3*df6.GEOM_R2.std())]
df6 = df6[np.abs(df6.GEOM_R3 - df6.GEOM_R3.mean()) <= (3*df6.GEOM_R3.std())]
df6 = df6[np.abs(df6.GEOM_R4 - df6.GEOM_R4.mean()) <= (3*df6.GEOM_R4.std())]
df6 = df6[np.abs(df6.CONTRUCTIONYEAR - df6.CONTRUCTIONYEAR.mean()) <= (3*df6.CONTRUCTIONYEAR.std())]
df6 = df6[np.abs(df6.MAXBUILDINGFLOOR - df6.MAXBUILDINGFLOOR.mean()) <= (3*df6.MAXBUILDINGFLOOR.std())]
df6 = df6[np.abs(df6.CADASTRALQUALITYID - df6.CADASTRALQUALITYID.mean()) <= (3*df6.CADASTRALQUALITYID.std())]

In [20]:
df = df0.append([df1, df2, df3, df4, df5, df6])

In [21]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [22]:
# Form np arrays of labels and features.
train_y = train_df.pop('CLASE')
#bool_train_labels = train_labels != 0
test_y = test_df.pop('CLASE')

train_X = train_df
test_X = test_df

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [24]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=train_X, label=train_y)
dtest = xgb.DMatrix(data=test_X)

In [25]:
params = {
    "booster":"gbtree",
    "silent":0,
    "scale_pos_weight": 1,
    "learning_rate": 0.01, ## empecé con 0.01.
    "colsample_bytree":0.3, ## empecé con 1. con 0,3 mejora
    "subsample": 1, #empecé con 0.8
    "n_estimators": 1000,
    "objective" : "multi:softprob",    
    "num_class" : 7,    
    "max_depth" : 32,   #Aumentando esto mejora
    "eta" : 0.3,    
    "gamma" : 5,
    "reg_alpha": 0.3
}
num_round = 20

In [26]:
bst = xgb.train(params, dtrain, num_round)

[22:38:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators, scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [27]:
pred = bst.predict(dtest)
#pred
best_preds = np.asarray([np.argmax(line) for line in pred])
#best_preds

In [28]:
from sklearn.metrics import precision_score
print(precision_score(test_y, best_preds, average='macro'))

0.9253407398995342


In [29]:
from sklearn.metrics import classification_report

print(classification_report(test_y, best_preds))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94        51
           1       0.85      0.70      0.77       792
           2       0.88      0.27      0.41       314
           3       0.97      0.39      0.55       281
           4       0.96      0.41      0.57       558
           5       0.93      1.00      0.96     16402
           6       0.97      0.15      0.25       386

    accuracy                           0.93     18784
   macro avg       0.93      0.55      0.64     18784
weighted avg       0.93      0.93      0.91     18784



In [30]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_y, best_preds)
cm

array([[   50,     1,     0,     0,     0,     0,     0],
       [    0,   553,     6,     0,     0,   232,     1],
       [    0,    42,    84,     1,     1,   185,     1],
       [    0,     7,     1,   109,     3,   161,     0],
       [    0,     5,     3,     2,   226,   322,     0],
       [    5,    19,     0,     0,     1, 16377,     0],
       [    0,    22,     1,     0,     4,   303,    56]], dtype=int64)

# ESTIMATE

In [50]:
stim = pd.read_csv("Estimar_UH2020.txt", sep = "|")
stim.dropna(inplace = True)

stim["CADASTRALQUALITYID"].replace("A", 10, inplace = True)
stim["CADASTRALQUALITYID"].replace("B", 11, inplace = True)
stim["CADASTRALQUALITYID"].replace("C", 12, inplace = True)

stim.iloc[:,1:3] = stim.iloc[:,1:3].astype("int32")
stim.iloc[:,3:52] = stim.iloc[:,3:52].astype("float32")
stim.iloc[:,52] = stim.iloc[:,52].astype("int32")
stim.iloc[:,53] = stim.iloc[:,53].astype("float32")
stim.iloc[:,54] = stim.iloc[:,54].astype("int32")

min_con= stim["CONTRUCTIONYEAR"].min()
stim["CONTRUCTIONYEAR"] = stim["CONTRUCTIONYEAR"] - min_con

In [51]:
stim_dist = pd.read_csv("estimarfeatures.csv")
stim_dist = stim_dist[["ID","Xpred","Ypred", "1000mresidential","1000mindustrial","1000mpublic","1000moffice","1000mother","1000mretail","1000magriculture",
                   "500mresidential","500mindustrial","500mpublic","500moffice","500mother","500mretail","500magriculture",
                   "250mresidential","250mindustrial","250mpublic","250moffice","250mother","250mretail","250magriculture",
                   "100mresidential","100mindustrial","100mpublic","100moffice","100mother","100mretail","100magriculture"]]
stim_dist.iloc[:,1:3] = stim_dist.iloc[:,1:3].astype("float32")
stim_dist.iloc[:,3:31] = stim_dist.iloc[:,3:31].astype("int8")

In [52]:
stim = pd.merge(stim, stim_dist, on = "ID")

In [53]:
stim.drop(stim[["Q_R_4_0_0","Q_R_4_1_0","Q_G_3_0_0","Q_G_3_1_0","Q_B_2_0_0","Q_B_2_1_0","Q_NIR_8_0_0","Q_NIR_8_1_0"]], axis = "columns", inplace = True)
stim.drop(stim[["ID"]], axis = "columns", inplace = True)

In [54]:
stim_X = np.array(stim)

In [55]:
stim_X = scaler.fit_transform(stim_X)
stim_X = xgb.DMatrix(data=stim_X)

In [56]:
est_y = bst.predict(stim_X)

In [57]:
best_est = np.asarray([np.argmax(line) for line in est_y])

In [58]:
df_est_y = pd.DataFrame(data=best_est)

In [59]:
print(df_est_y[0].value_counts())

5    5365
1     220
0      18
4       8
Name: 0, dtype: int64


###  

###  

###  

In [110]:
from imblearn.over_sampling import RandomOverSampler
train_X1, train_y1 = RandomOverSampler().fit_resample(train_X, train_y)
test_X1, test_y1 = RandomOverSampler().fit_resample(test_X, test_y)

Using TensorFlow backend.


In [118]:
from imblearn.over_sampling import ADASYN
train_X1, train_y1 = ADASYN().fit_resample(train_X, train_y)
test_X1, test_y1 = ADASYN().fit_resample(test_X, test_y)

In [126]:
from imblearn.over_sampling import SMOTE
train_X1, train_y1 = SMOTE().fit_resample(train_X, train_y)
test_X1, test_y1 = SMOTE().fit_resample(test_X, test_y)

In [127]:
import xgboost as xgb

dtrain1 = xgb.DMatrix(data=train_X1, label=train_y1)
dtest1 = xgb.DMatrix(data=test_X1)

In [134]:
params1 = {
    "booster":"gbtree",
    "silent":0,
    "scale_pos_weight": 1,
    "learning_rate": 0.01, ## empecé con 0.01.
    "colsample_bytree":0.3, ## empecé con 1. con 0,3 mejora
    "subsample": 1, #empecé con 0.8
    "n_estimators": 1000,
    "objective" : "multi:softprob",    
    "num_class" : 7,    
    "max_depth" : 32,   #Aumentando esto mejora
    "eta" : 0.3,    
    "gamma" : 5,
    "reg_alpha": 0.3,
    "max_delta_step": 10
}
num_round = 20

In [135]:
bst = xgb.train(params1, dtrain1, num_round)

[19:50:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators, scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [136]:
pred1 = bst.predict(dtest1)
#pred
best_preds1 = np.asarray([np.argmax(line) for line in pred1])
#best_preds

In [137]:
from sklearn.metrics import precision_score
print(precision_score(test_y1, best_preds1, average='macro'))

0.6327638771170168


In [138]:
from sklearn.metrics import classification_report

print(classification_report(test_y1, best_preds1))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91     18025
           1       0.62      0.67      0.65     18025
           2       0.48      0.46      0.47     18025
           3       0.61      0.68      0.64     18025
           4       0.52      0.47      0.49     18025
           5       0.74      0.90      0.82     18025
           6       0.53      0.41      0.46     18025

    accuracy                           0.64    126175
   macro avg       0.63      0.64      0.63    126175
weighted avg       0.63      0.64      0.63    126175



In [139]:
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(test_y1, best_preds1)
cm1

array([[16090,   561,   356,   175,   618,    21,   204],
       [  289, 12018,  1676,  1167,   809,   856,  1210],
       [  373,  3155,  8223,  1360,  2014,   998,  1902],
       [  167,   835,   868, 12328,  2038,   660,  1129],
       [  167,   761,  2723,  3063,  8446,   928,  1937],
       [   36,   176,   478,   210,   608, 16305,   212],
       [  239,  1734,  2855,  2004,  1625,  2207,  7361]], dtype=int64)